In [1]:
from transformers import GPT2LMHeadModel, AutoTokenizer

import torch
import numpy as np
import pandas as pd

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
model_name = 'metamath/kogpt2-inst-newstitle'

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    pad_token='<pad>',
    eos_token='</s>')

tokenizer_config.json:   0%|          | 0.00/64.2k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/891k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/496k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.25M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

In [6]:
model = GPT2LMHeadModel.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [7]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [11]:
# 학습하기전 각종 뉴스 분야 키워드를 입력해서 제목을 생성
prompts = ["IT과학 <|> ", "경제 <|> ", "사회 <|> ", "생활문화 <|> ", "세계 <|> ", "스포츠 <|> ", "정치 <|> "]

for prompt in prompts:
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        generated_ids = model.generate(
            input_ids,
            do_sample=True,
            min_length=10,
            max_length=50,
            repetition_penalty=1.5,
            temperature=0.9,
        )
        print(tokenizer.decode([el.item() for el in generated_ids[0]]))
        print("---------------------------------------------------------")


IT과학 <|> 롯데정보통신 VR 콘텐츠 서비스 출시</s>
---------------------------------------------------------
경제 <|> 롯데마트 서울 소공동 명품관 PB스토어 오픈...우리카드 신한카드 결제 고객 대상 이벤트</s>
---------------------------------------------------------
사회 <|> 美 민주콩고서 대통령 탄핵안 부결...공화 대선후보 지지율 20%대 하락</s>
---------------------------------------------------------
생활문화 <|> 퓰리처상 수상작들 속 아름다운 것들</s>
---------------------------------------------------------
세계 <|> 英 최대 정치지중해 언론사 홍콩서 철수...내년 1월1일부터 합법화종합</s>
---------------------------------------------------------
스포츠 <|> 넥센 SK kt 꺾고 5연승 질주 행진</s>
---------------------------------------------------------
정치 <|> 朴대통령 北 핵실험에 강력 규탄...한반도 안보위협 고조종합</s>
---------------------------------------------------------
